__No te olvides de rellenar esto:__

- Número de grupo: 10
- Nombre de los integrantes del grupo: Francisco de Borja Lozano del Moral, Manuel Ortega Salvador

# Práctica 1

> __Fecha de entrega: 11 de abril de 2021__


## Parte 2: similitud semántica

Una de las grandes ventajas de las representaciones estructuradas es que podemos aprovechar su estructura para calcular similitudes semánticas entre las entidades. En esta ocasión vamos a cacular la similitud entre dos conceptos como:

$$Sim(A, B) = \frac{\delta(root, C)}{\delta(root, C) + \delta(C, A) + \delta(C, B)}$$

siendo:

- $\delta(X, Y)$ el __mínimo__ número de aristas que conecta A y B, siendo A más general que B.
- $C = LCS(A, B)$ el concepto más específico de la jerarquía que es más general que A y B (_least common subsummer_).

La idea tras esta similitud queda reflejada en la siguiente imagen:

<img src="sim.png" alt="Similitud" style="width: 300px;"/>

En la práctica pueden existir distintos conceptos C que cumplen la definición de _least common subsummer_ de A y B por lo que es necesario definir cuál de ellos vamos a utilizar. En nuestro caso seleccionaremos __uno de los que maximiza el valor de similitud__. 

### 1) Obtener la taxonomía con la que vamos a trabajar

Utiliza el [punto el acceso](https://query.wikidata.org/) SPARQL de Wikidata para ejecutar una consulta que devuelva todos los pares de entidades $(x, y)$ tal que $x$ es subconcepto directo de $y$ y ambos son un tipos de [instrumentos musicales (Q34379)](https://www.wikidata.org/wiki/Q34379). Debes recuperar tantos las URIs de la entidades como sus etiquetas.

Escribe en la siguiente celda la consulta que has utilizado comentada adecuadamente.

A continuación descarga todas las respuestas en formato _Archivo JSON_ y guardalo en el mismo directorio de la práctica.

_Nota: en el momento de realizar esta práctica obtuve 4727 resultados pero el número puede variar al ser Wikidata una base de conocimiento dinámica._

### 2) Cargar la taxonomía en memoria

Vamos a cargar la taxonomía de clases en memoria para poder operar con ella. Representaremos la jerarquía de lcases mediantes las siguientes estructuras:

- Un diccionario que asocia a cada identificador su etiqueta (por ejemplo 'Q34379' -> 'musical instrument')
- Un diccionario que asocia cada clase con sus subclases directas (por ejemplo 'Q695269' -> {'Q25630013', 'Q3388256', 'Q524526', 'Q846109', 'Q960389'} )
- Un diccionario que asocia cada clase con sus superclases directas (por ejemplo 'Q34379' -> {'Q1879241', 'Q54820129'} )

Tienes libertad para elegir cómo quieres representar la taxonomía en Python:

- Puedes usar una clase. En ese caso tendrás que ir añadiendo métodos a la clase para completar cada uno de los apartados de la práctica. Escribe el código de la clase en una única celda y utiliza los métodos que necesites en cada uno de los apartados.
- Puedes usar 3 variables globales para representar la taxonomía. En ese caso deberás escribir las operaciones como funciones en cada uno de los apartados de la práctica.

En cualquier caso recuerda documentar adecuadamente el código y trata de que sea sencillo de entender.

Crea una operación _load_ que reciba el nombre del fichero json y cargue el grafo en memoria usando las estructuras anteriores.

```python
import json

with open(filename) as f:
    data = json.load(f)
```

In [1]:
import json

#Diccionario identificador:etiqueta
diccionario_etiqueta = {}
#Diccionario identificador:subclases
diccionario_subclases = {}
#Diccionario identificador:superclases
diccionario_superclases = {}
#Primero metemos instrumentos musicales, ya que nunca va a aparecer como subclase de otra clase
superclase_raiz = set()
diccionario_superclases.update({"Q34379":superclase_raiz})

def load(filename):

    with open(filename, encoding='utf-8') as f:
        data = json.load(f)
        
    for item in data:
        #Cogemos el identificador, pero quitamos la parte del enlace a wikidata por más limpieza
        id_x = item.get("x").replace("http://www.wikidata.org/entity/",'')
        label_x = item.get("xLabel")
        diccionario_etiqueta.update({id_x:label_x})
        id_y = item.get("y").replace("http://www.wikidata.org/entity/",'')
        label_y = item.get("yLabel")
        diccionario_etiqueta.update({id_y:label_y})
        
        #Ahora llenamos el diccionario de superclases
        if id_x in diccionario_superclases:
            superclase_x = diccionario_superclases.get(id_x)
            superclase_x.update({id_y})
        else:
            superclase_x = {id_y}
        diccionario_superclases.update({id_x:superclase_x})
        
        #Ahora llenamos el diccionario de subclases
        if id_y in diccionario_subclases:
            subclase_y = diccionario_subclases.get(id_y)
            subclase_y.update({id_x})
        else:
            subclase_y = {id_x}
        diccionario_subclases.update({id_y:subclase_y})
        
        #Para que queden añadidos aquellos que no tienen ninguna subclase
        if id_x not in diccionario_subclases:
            subclase_x = set()
            diccionario_subclases.update({id_x:subclase_x})
        

Comprobamos primero que los tres diccionarios tienen el mismo número de elementos.

In [2]:
load("query.json")
len(diccionario_etiqueta)

4166

In [3]:
len(diccionario_subclases)

4166

In [4]:
len(diccionario_superclases)

4166

Aquí comprobamos que se han construido bien los diccionarios.

In [5]:
diccionario_superclases

{'Q34379': set(),
 'Q2046356': {'Q133163'},
 'Q3144475': {'Q1071649', 'Q133163'},
 'Q7536920': {'Q133163'},
 'Q68647735': {'Q133163', 'Q185003'},
 'Q1428415': {'Q133163'},
 'Q94955292': {'Q133163'},
 'Q5280486': {'Q12639', 'Q133163'},
 'Q943438': {'Q12639', 'Q133163'},
 'Q609599': {'Q133163'},
 'Q2781273': {'Q133163', 'Q55723221'},
 'Q270200': {'Q133163', 'Q1575231'},
 'Q12486450': {'Q133163'},
 'Q11910499': {'Q133163'},
 'Q5017970': {'Q133163'},
 'Q1372823': {'Q105738'},
 'Q11913230': {'Q133163'},
 'Q128309': {'Q133163', 'Q55723238'},
 'Q194492': {'Q133163'},
 'Q7821057': {'Q133163'},
 'Q5959276': {'Q133163'},
 'Q6138484': {'Q133163'},
 'Q7198871': {'Q133163'},
 'Q27924148': {'Q133163'},
 'Q3998771': {'Q133163'},
 'Q11926747': {'Q133163'},
 'Q11926749': {'Q133163'},
 'Q3305735': {'Q133163'},
 'Q65129374': {'Q133163', 'Q34379'},
 'Q19588483': {'Q133163', 'Q212892'},
 'Q2046658': {'Q133163', 'Q4863335'},
 'Q93985': {'Q133163'},
 'Q30674028': {'Q133163'},
 'Q1458221': {'Q133163'},
 'Q119

In [6]:
diccionario_etiqueta

{'Q2046356': 'pahu',
 'Q133163': 'instrumento de percusión',
 'Q3144475': 'hyōshigi',
 'Q7536920': 'skull',
 'Q68647735': 'aman huur',
 'Q1428415': 'Kpanlogo',
 'Q94955292': 'Foot percussion',
 'Q5280486': 'idiófono golpeado',
 'Q943438': 'chalaparta',
 'Q609599': 'mizhavu',
 'Q2781273': 'tambori',
 'Q270200': 'conga',
 'Q12486450': 'Q12486450',
 'Q11910499': 'Q11910499',
 'Q5017970': 'Caja vallenata',
 'Q1372823': 'instrumento electromecánico',
 'Q105738': 'electrófono',
 'Q11913230': 'Q11913230',
 'Q128309': 'batería',
 'Q194492': 'Gamelan',
 'Q7821057': 'tongatong',
 'Q5959276': 'DAMMAM',
 'Q6138484': 'Tambor doble (México)',
 'Q7198871': 'pitched percussion instrument',
 'Q27924148': 'Q27924148',
 'Q3998771': 'stomp box',
 'Q11926747': 'Q11926747',
 'Q11926749': 'Q11926749',
 'Q3305735': 'Tom de piso',
 'Q65129374': 'Q65129374',
 'Q19588483': 'struck membranophones',
 'Q2046658': 'Pakhawaj',
 'Q93985': 'crash cymbal',
 'Q30674028': 'Davandai',
 'Q1458221': 'Tambora dominicana',
 'Q

In [8]:
diccionario_subclases

{'Q133163': {'Q104363',
  'Q105379653',
  'Q1189515',
  'Q11910499',
  'Q11913230',
  'Q11926747',
  'Q11926749',
  'Q11941932',
  'Q11950748',
  'Q11955385',
  'Q12486450',
  'Q12690320',
  'Q128309',
  'Q13219842',
  'Q133722',
  'Q1410790',
  'Q1428415',
  'Q1437089',
  'Q1439716',
  'Q1458221',
  'Q14640031',
  'Q1644824',
  'Q1661583',
  'Q17062763',
  'Q17138362',
  'Q17621647',
  'Q1898734',
  'Q194492',
  'Q19588483',
  'Q19754610',
  'Q2046356',
  'Q2046658',
  'Q21087039',
  'Q21087294',
  'Q21091809',
  'Q21099529',
  'Q2128998',
  'Q2360352',
  'Q2368593',
  'Q25432192',
  'Q270200',
  'Q2781273',
  'Q27924148',
  'Q28497151',
  'Q2898290',
  'Q30674028',
  'Q3083684',
  'Q3144475',
  'Q3193614',
  'Q3275471',
  'Q3305735',
  'Q3522389',
  'Q354765',
  'Q3574205',
  'Q3623050',
  'Q395212',
  'Q3998771',
  'Q4422479',
  'Q4465189',
  'Q4848966',
  'Q4898607',
  'Q4986147',
  'Q5017970',
  'Q5032021',
  'Q5280486',
  'Q56643162',
  'Q5719092',
  'Q5732744',
  'Q5765986',
  '

### 3) Imprimir un subárbol de la taxonomía

Crea una operación _print_tree_ que imprimir la jerarquía de clases a partir de un concepto y hasta un nivel de profundidad determinado.

Por ejemplo, a continuación podemos ver el principio de la jerarquía de [voces](https://www.wikidata.org/wiki/Q17172850) con 3 niveles de profundidad:

```
0 voz (Q17172850)
  1 operatic vocal (Q101436564)
  1 alto (Q6983813)
   2 mezzosoprano ligera (Q6012300)
   2 boy alto (Q53395277)
   2 alto castrato (Q53395016)
   2 contralto (Q37137)
  1 contralto (Q37137)
   2 contralto cómica (Q5785182)
   2 lyric contralto (Q54635214)
   2 Tenorino (Q6141663)
   2 contralto de coloratura (Q54635184)
   2 deep contralto (Q54635335)
   2 contralto dramática (Q5785183)
  1 bajo (Q27911)
   2 heavy acting bass (Q54636271)
   2 bajo profundo (Q2532487)
   2 bajo buffo (Q1002146)
   ...
```

Como ocurre en todas las grandes bases de conocimiento, dentro de Wikidata hay información que no ha sido bien introducida o está mal clasificada. ¿Puedes encontrar algún ejemplo concreto dentro de la jerarquía de instrumentos?

In [9]:
#Función recursiva que va generando el string de texto que representa la jerarquía de clases
def print_nivel(concepto, nivel_hasta, nivel_actual):
    arbol = ""
    if nivel_actual < nivel_hasta:
        for i in range(0,nivel_actual):
            arbol += "  "
        arbol += str(nivel_actual) + ' ' + diccionario_etiqueta.get(concepto) + ' ' + '(' + concepto + ')' + '\n'
        for subclase in diccionario_subclases.get(concepto):
            arbol += print_nivel(subclase, nivel_hasta, nivel_actual+1)
            
    return arbol   

def print_tree(concepto, nivel):
    arbol = print_nivel(concepto, nivel, 0)
    print(arbol)

Probamos para la jerarquía de _voces_ (Q17172850).

In [10]:
print_tree("Q17172850", 3)

0 voz (Q17172850)
  1 bajo (Q27911)
    2 bajo-barítono (Q810480)
    2 octavist (Q7082656)
    2 high bass (Q54636007)
    2 bajo cantante (Q3636053)
    2 bajo caractère (Q20638448)
    2 grave (Q5885030)
    2 heavy acting bass (Q54636271)
    2 dramatic high bass (Q54636036)
    2 serious bass (Q54636068)
    2 bajo profundo (Q2532487)
    2 Bass bourdon (Q64363543)
    2 bajo buffo (Q1002146)
  1 alto (Q6983813)
    2 contralto (Q37137)
    2 boy alto (Q53395277)
    2 alto castrato (Q53395016)
    2 mezzosoprano ligera (Q6012300)
  1 Voz blanca (Q6164901)
    2 child soprano (Q53829839)
  1 Xöömej (Q1067089)
  1 backing vocal (Q60396389)
  1 operatic vocal (Q101436564)
  1 mezzosoprano (Q186506)
    2 altmezzo (Q682525)
    2 mezzo-soprano castrato (Q54634945)
    2 sopranista (Q1999862)
    2 Q1300059 (Q1300059)
    2 mezzosoprano dramática (Q6012297)
    2 coloratura mezzo-soprano (Q54634572)
    2 character mezzo-soprano (Q54634862)
    2 mezzosoprano ligera (Q54634726)
    2 

Probamos ahora con la jerarquía de _instrumentos_ (Q34379).

In [11]:
print_tree("Q34379", 8)

0 instrumento musical (Q34379)
  1 Boomwhacker (Q893045)
  1 Caña Rociera (Q1052089)
  1 Harmonizer (Q3623161)
  1 Q2958769 (Q2958769)
  1 kutiyapi (Q12485397)
  1 Q28170750 (Q28170750)
  1 Halam (Q5891381)
  1 Roland SP-808 (Q65067370)
  1 Cajita rítmica afroperuana (Q5739510)
  1 Q27510104 (Q27510104)
  1 Q28192975 (Q28192975)
  1 Q46996068 (Q46996068)
  1 Uruttu Chenda (Q17070147)
  1 Violí de bufa (Q11954880)
  1 Adiaphone (Q356923)
  1 Q20106313 (Q20106313)
  1 marching instrument (Q16918224)
    2 marching brass instrument (Q12486669)
      3 Marching euphonium (Q16253812)
  1 Akete (Q3346567)
  1 Rigo-rago (Q20103238)
  1 Fish-drum (Q5454461)
  1 improvised musical instrument (Q56862947)
    2 Blown bottle (Q4928615)
  1 pi nai (Q16304285)
  1 talempong (Q7679165)
  1 Hom (Q16919362)
  1 Q25395631 (Q25395631)
  1 Zendrum (Q3040117)
  1 Saxtuba (Q7428890)
  1 Nagara (Q2510552)
  1 Cataquí (Q2551648)
  1 galgo (Q5518524)
  1 Combo organ (Q5150931)
  1 Q3868722 (Q3868722)
  1 idióf

Si seguimos el camino instrumento musical -> canto -> declamación -> atributo heráldico, vemos sus hijos no tienen nada que ver con instrumentos musicales, por lo que es un ejemplo de información de Wikidata mal clasificada.

### 4) Obtener los LCS

Crea una operación _lcs_ que devuelva todos los LCS de dos conceptos determinados. Recuerda que un concepto C es LCS(A, B) si es más general que ambos y no se puede especializar más sin dejar de serlo.

Para implementarlo seguramente te resulte útil tener otro método que devuelva todos los conceptos más generales que uno dado. _Pista: es fácil de implementar usando operaciones entre conjuntos_. 

Ejemplos:

```
mezzosoprano dramática (Q6012297), mezzosoprano ligera (Q54634726), mezzosoprano (Q186506)
LCS('Q6012297', 'Q54634726') = {'Q186506'}

grave (Q5885030), mezzosoprano ligera (Q6012300), voz (Q17172850)
LCS('Q5885030', 'Q6012300') = {'Q17172850'}

tenor (Q27914)
LCS('Q27914', 'Q27914') = {'Q27914'}

viola eléctrica (Q15336282), bajo eléctrico (Q64166304), instrumento de cuerda (Q1798603), electrófono (Q105738), necked box lutes (Q55724840)
LCS('Q15336282', 'Q64166304') = {'Q55724840', 'Q105738', 'Q1798603}
```

In [12]:
#Dado un concepto, devuelve todos los conceptos más generales que él
def conceptos_mas_generales(concepto):
    conceptos_mas_gen = set()
    set_exploracion = {concepto}
    while set_exploracion:
        item = set_exploracion.pop()
        conceptos_mas_gen = conceptos_mas_gen | diccionario_superclases.get(item)
        set_exploracion = set_exploracion | diccionario_superclases.get(item)
        
    return conceptos_mas_gen

def LCS(concepto1, concepto2):
    #Si son iguales, él mismo será el LCS
    if concepto1 == concepto2:
        return {concepto1}
    else:
        #Primero calculamos los más generales de cada uno, y le añadimos a ellos mismos para el caso en que uno sea superclase
        #(directa o indirectamente) del otro
        mas_gen_A = conceptos_mas_generales(concepto1)
        mas_gen_A.add(concepto1)
        mas_gen_B = conceptos_mas_generales(concepto2)
        mas_gen_B.add(concepto2)
        #Hacemos intersección de los más generales de ambos para ver los comunes
        set_lcs = mas_gen_A & mas_gen_B
        #Como queremos que sean los comunes más específicos, le quitamos los que son más generales que los comunes
        #ya encontrados, en caso de que hubiera alguno en el set.
        for item in set_lcs:
            set_lcs = set_lcs - conceptos_mas_generales(item)
            
        return set_lcs
        

Probamos las funciones.

In [13]:
conceptos_mas_generales("Q932855")

{'Q1051772',
 'Q1798603',
 'Q1808578',
 'Q19588495',
 'Q230262',
 'Q30038759',
 'Q3087584',
 'Q34379',
 'Q46185',
 'Q55724833',
 'Q55724840',
 'Q57306162',
 'Q6607',
 'Q810447',
 'Q98329515'}

In [14]:
LCS("Q6012297", "Q54634726")

{'Q186506'}

In [15]:
LCS('Q5885030', 'Q6012300')

{'Q17172850'}

In [16]:
LCS('Q27914', 'Q27914')

{'Q27914'}

In [17]:
LCS('Q15336282', 'Q64166304')

{'Q105738', 'Q1798603', 'Q55724840'}

### 5) Obtener caminos mínimos

Crea una operación _path_ que calcule el camino mínimo entre dos conceptos A y B siendo A más o igual de general que B. Como la taxonomía no tiene ciclos puedes implementarlo como una búsqueda en profunidad. Ten en cuenta que los caminos sólo pueden contener conceptos más específicos o iguales a A y más generales o iguales a B.

Ejemplos:

```
path('Q186506', 'Q54634726') = [mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

path('Q17172850', 'Q6012300') = [voz (Q17172850), alto (Q6983813), mezzosoprano ligera (Q6012300)]

path('Q27914', 'Q27914') = [tenor (Q27914)]

path('Q34379', 'Q105738') = [instrumento musical (Q34379), cordófono (Q1051772), composite chordophones (Q19588495), lutes (Q1808578), handle lutes (Q30038759), necked lutes (Q55724833), necked box lutes (Q55724840)]
 ```

In [18]:
def dfs_path(concepto_actual, concepto2, conceptos_mas_generales_B):
    if concepto_actual == concepto2:
        return [diccionario_etiqueta.get(concepto2) + ' ' + '(' + concepto2 + ')']
    else:
        subclases_concepto_actual = diccionario_subclases.get(concepto_actual)
        #El camino solo puede contener conceptos más específicos del conpcepto actual y más generales que B
        hijos_dfs = subclases_concepto_actual & conceptos_mas_generales_B
        min_path = []
        #Buscamos para cada uno de los hijos válidos el camino más corto
        for hijo in hijos_dfs:
            next_path = dfs_path(hijo, concepto2, conceptos_mas_generales_B)
            #Si min_path aún no tiene valor o el encontrado es mejor que el anterior mejor, se actualiza
            if (len(min_path) == 0) or (len(min_path) > 0 and len(next_path) < len(min_path)):
                min_path = next_path
        
        return [diccionario_etiqueta.get(concepto_actual) + ' ' + '(' + concepto_actual + ')'] + min_path
            
        
    
def path(concepto1, concepto2):
    camino = [diccionario_etiqueta.get(concepto1) + ' ' + '(' + concepto1 + ')']
    mas_general_B = conceptos_mas_generales(concepto2)
    #Metemos también el segundo concepto para que pueda ser seleccionado como hijo al buscar el camino
    mas_general_B.add(concepto2)
    return dfs_path(concepto1, concepto2, mas_general_B)
        

In [19]:
path('Q186506', 'Q54634726')

['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']

In [20]:
path('Q17172850', 'Q6012300')

['voz (Q17172850)', 'alto (Q6983813)', 'mezzosoprano ligera (Q6012300)']

In [21]:
path('Q27914', 'Q27914')

['tenor (Q27914)']

El resultado del ejemplo de abajo no coincide con el que aparece en el enunciado, pero si nos fijamos en el subárbol de la taxonomía desde _instrumento musical_ hecho en un ejemplo más arriba, podemos ver que _electrófono_ es hijo directo de _instrumento musical_ (recordemos que Wikidata se está actualizando constantemente).

In [22]:
path('Q34379', 'Q105738')

['instrumento musical (Q34379)', 'electrófono (Q105738)']

In [23]:
path('Q34379','Q76938365')

['instrumento musical (Q34379)',
 'aerófono (Q659216)',
 'free aerophone (Q50840728)',
 'interruptive free aerophone (Q54819588)',
 'sirena (Q205234)',
 'directional siren (Q25111662)',
 'omnidirectional siren (Q76923428)',
 'Model 5 (Q76938365)']

### 6) Calcular la similitud

Implementa una operación _similarity_ que calcule la similtud entre dos conceptos. Debe devolver tanto el valor númerico de similitud como los caminos desde la raiz al LCS y desde el LCS a cada uno de los dos conceptos.

Ten en cuenta que debes usar un LCS que maximice el valor de similitud. Si la información de Wikidata no ha cambiado, los valores de similitud deberían coincidir con los que aparecen en los ejemplos pero los caminos no tienen por qué. Y recuerda que no es lo mismo el números de aristas de un camino que el número de nodos del camino.

Ejemplos:

```
similarity('Q6012297', 'Q54634726')
0.5
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano dramática (Q6012297)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q186506', 'Q54634726')
0.6666666666666666
[instrumento musical (Q34379), voz (Q17172850), mezzosoprano (Q186506)]
[mezzosoprano (Q186506)]
[mezzosoprano (Q186506), mezzosoprano ligera (Q54634726)]

similarity('Q27914', 'Q27914')
1.0
[instrumento musical (Q34379), voz (Q17172850), high voice (Q98116969), tenor (Q27914)]
[tenor (Q27914)]
[tenor (Q27914)]

similarity('Q76239', 'Q78987')
0.42857142857142855
[instrumento musical (Q34379), cordófono (Q1051772), instrumento de cuerda (Q1798603), instrumento de cuerda pulsada (Q230262)]
[instrumento de cuerda pulsada (Q230262), cítara (Q76239)]
[instrumento de cuerda pulsada (Q230262), plucked necked box lutes (Q57306162), guitarra (Q6607), guitarra eléctrica (Q78987)]
```

In [23]:
 def similarity(concepto1, concepto2):
    lcs_set = LCS(concepto1, concepto2)
    raiz = 'Q34379'
    max_similarity = 0
    max_path = []
    for lcs in lcs_set:
        path_root_lcs = path(raiz, lcs)
        path_lcs_A = path(lcs, concepto1)
        path_lcs_B = path(lcs, concepto2)
        this_similarity = (len(path_root_lcs)-1)/((len(path_root_lcs)-1)+(len(path_lcs_A)-1)+(len(path_lcs_B)-1))
        if this_similarity >= max_similarity:
            max_similarity = this_similarity
            max_path = [path_root_lcs, path_lcs_A, path_lcs_B]
    
    print(max_similarity)
    print(max_path[0])
    print(max_path[1])
    print(max_path[2])
    

In [24]:
similarity('Q6012297', 'Q54634726')

0.5
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano dramática (Q6012297)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']


In [25]:
similarity('Q186506', 'Q54634726')

0.6666666666666666
['instrumento musical (Q34379)', 'voz (Q17172850)', 'mezzosoprano (Q186506)']
['mezzosoprano (Q186506)']
['mezzosoprano (Q186506)', 'mezzosoprano ligera (Q54634726)']


In [26]:
similarity('Q27914', 'Q27914')

1.0
['instrumento musical (Q34379)', 'voz (Q17172850)', 'high voice (Q98116969)', 'tenor (Q27914)']
['tenor (Q27914)']
['tenor (Q27914)']


In [27]:
similarity('Q76239', 'Q78987')

0.42857142857142855
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


### 7) Análisis de las similitudes

Calcula la similitud 2 a 2 de los siguientes instrumentos y explica razonadamente si los valores obtenidos tienen sentido de acuerdo a tu intuición sobre si se parecen o no.

```
piano (Q5994), guitarra (Q6607), guitarra eléctrica (Q78987), flauta (Q11405), trompeta (Q8338)
```

In [28]:
similarity('Q5994', 'Q6607')

0.2727272727272727
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)', 'board zithers (Q50829016)', 'true board zithers (Q55724736)', 'true board zithers with resonator (Q55724742)', 'true board zithers with resonator box (Q4951628)', 'piano (Q5994)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']


In [29]:
similarity('Q5994', 'Q78987')

0.25
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)']
['instrumento de cuerda pulsada (Q230262)', 'cítara (Q76239)', 'board zithers (Q50829016)', 'true board zithers (Q55724736)', 'true board zithers with resonator (Q55724742)', 'true board zithers with resonator box (Q4951628)', 'piano (Q5994)']
['instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


En este primer ejemplo vemos que guitarra y piano tienen una cierta similitud, ya que ambos son instrumentos de cuerda, concretamente de cuerda pulsada, cosa que se ve en el camino desde la raíz hasta su LCS.

Además vemos que guitarra eléctrica tiene algo de menor semejanza con piano que guitarra, cosa que tiene sentido al ser esta un tipo de guitarra (se ve que los caminos son los mismos, añadiendo simplemente guitarra eléctrica al final del camino en su ejemplo)

In [30]:
similarity('Q5994', 'Q11405')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento-madera (Q181247)', 'flauta (Q11405)']


In [32]:
similarity('Q5994', 'Q8338')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'teclado (Q52954)', 'piano (Q5994)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


Aquí vemos que la semejanza de piano con estos dos instrumentos es 0, ya que la raíz del árbol es su único ancestro común. Esto tiene sentido ya que piano es un instrumento de cuerda, mientras que trompeta y flauta son de viento, por lo que ninguno de ellos tienen nada en común respecto a la forma en que se tocan, ni a los elementos que los componen.

In [33]:
similarity('Q6607', 'Q78987')

0.8333333333333334
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']
['guitarra (Q6607)']
['guitarra (Q6607)', 'guitarra eléctrica (Q78987)']


Tiene sentido que guitarra y guitarra eléctrica tengan una similitud muy alta, ya que este último concepto es hijo directo del primero, al ser un tipo concreto.

In [34]:
similarity('Q6607', 'Q8338')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'cordófono (Q1051772)', 'instrumento de cuerda (Q1798603)', 'instrumento de cuerda pulsada (Q230262)', 'plucked necked box lutes (Q57306162)', 'guitarra (Q6607)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


In [35]:
similarity('Q78987', 'Q11405')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'electrófono (Q105738)', 'electro-acoustic instruments (Q50821977)', 'electro-acoustic chordophones (Q55738960)', 'guitarra eléctrica (Q78987)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento-madera (Q181247)', 'flauta (Q11405)']


In [36]:
similarity('Q78987', 'Q8338')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'electrófono (Q105738)', 'electro-acoustic instruments (Q50821977)', 'electro-acoustic chordophones (Q55738960)', 'guitarra eléctrica (Q78987)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


In [37]:
similarity('Q78987', 'Q8338')

0.0
['instrumento musical (Q34379)']
['instrumento musical (Q34379)', 'electrófono (Q105738)', 'electro-acoustic instruments (Q50821977)', 'electro-acoustic chordophones (Q55738960)', 'guitarra eléctrica (Q78987)']
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


Al igual que sucedía con piano, guitarra por ser intrumento de cuerda no tiene nada de similitud con instrumentos de viento como la trompeta y la flauta. Por ser guitarra eléctrica tan semejante a guitarra, sucede lo mismo.

In [38]:
similarity('Q11405', 'Q8338')

0.4
['instrumento musical (Q34379)', 'aerófono (Q659216)', 'instrumento de viento (Q173453)']
['instrumento de viento (Q173453)', 'flauta (Q11405)']
['instrumento de viento (Q173453)', 'instrumento de viento-metal (Q180744)', 'trompeta (Q8338)']


Primero de todo apreciamos que la similitud entre trompeta y flauta es mayor que la que teníamos entre guitarra y piano. Esto tiene sentido porque, aunque piano y guitarra sean instrumentos de cuerda, la forma en la que se tocan y los materiales que los constituyen son muy diferentes. Sin embargo, la flauta y la trompeta se tocan de una manera muy similar, y como vemos en el camino que se ha seguido, la única distinción que se hace entre ambas es el material del que están hechas.